# 判断新闻的来源是否抄袭

In [1]:
import pandas as pd
import numpy as np
import jieba
import re

# 数据预处理

In [2]:
fpath = 'C:/Users/Jing/downloads/sqlResult_1558435.csv'

In [3]:
corpus = pd.read_csv(fpath, encoding='gb18030')

In [102]:
corpus.head(3)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm


In [5]:
len(corpus)

89611

In [6]:
news_corpus = corpus.dropna(subset=['content'])  # 去掉空新闻

In [12]:
news_corpus['source'] = news_corpus['source'].fillna('') # 补充缺省值

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
neg_data = news_corpus[news_corpus['source'].str.contains('新华社') == 0]
pos_data = news_corpus[news_corpus['source'].str.contains('新华社')]
n_sample = 16600  #样本总数

neg_data = neg_data.sample(n = (int) (n_sample / 2))
pos_data = pos_data.sample(n = (int) (n_sample / 2))

sample = pos_data.append(neg_data)


In [17]:
len(sample)

16600

In [18]:
def label(line):
    if line['source'] == '新华社': la = 1
    else: la = 0
    return pd.Series([line['content'], la], index=['content', 'label'])
    

In [19]:
news_data = sample.apply(label, axis=1)

In [20]:
news_list = news_data.content.tolist()
y = news_data.label.values.astype(np.int)

In [21]:
y.shape

(16600,)

# tfidf 文本向量化

In [22]:
from tqdm import tqdm

In [23]:
news_cut = []
mask = []
for sentence in tqdm(news_list):
    if not isinstance(sentence, str):
        mask.append(False)
        continue
    mask.append(True)
    sentence = ''.join(re.findall(r'\w+', sentence))
    news_cut.append(" ".join(jieba.cut(sentence)))
    
len(news_cut)
    

  0%|                                                | 0/16600 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Jing\AppData\Local\Temp\jieba.cache
Loading model cost 2.906 seconds.
Prefix dict has been built succesfully.
100%|████████████████████████████████████| 16600/16600 [03:53<00:00, 71.23it/s]


16600

In [24]:
len(mask)

16600

In [25]:
news_cut[0]

'新华社 照片 外代 2017 年 5 月 14 日 n 外代 二线 葡萄牙 歌手 索 布拉 尔 夺得 欧洲 电视 歌曲 大赛 冠军 n5 月 13 日 在 乌克兰 首都 基辅 国际展览中心 葡萄牙 歌手 萨尔瓦多 索 布拉 尔 在 摘取 大赛 桂冠 后 庆贺 n 当日 第 62 届 欧洲 电视 歌曲 大赛 总决赛 在 乌克兰 基辅 落下 帷幕 葡萄牙 歌手 萨尔瓦多 索 布拉 尔 夺得 冠军 n 新华社 路透'

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
vector = TfidfVectorizer(max_features = 200)
X = vector.fit_transform(news_cut)

In [28]:
X.shape

(16600, 200)

# KNN MODEL and Logistic Regression

In [29]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=4, test_size=0.15)

In [34]:
X_train.shape, X_test.shape

((14110, 200), (2490, 200))

KNN k_n=5

In [35]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, random_state=42, test_size=0.2)

In [36]:
x_train.shape, x_valid.shape

((11288, 200), (2822, 200))

In [37]:
knn = KNeighborsClassifier(n_jobs=-1)
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

In [ ]:
def get_scores(x_valid, y_valid):
    val = knn.score(x_valid, y_valid)
    y_predict = knn.predict(x_valid)
    
    prec = matrics.precision_score(y_valid, y_predict)
    recall = matrics.recall_score(y_valid, y_predict)
    f1 = matrics.fi_score(y_valid, y_predict)
    auc = matrics.roc_auc_score(y_valid, y_predict)
    
    return
    
    


In [38]:
knn.score(x_valid, y_valid)

0.8561304039688165

In [39]:
y_predict = knn.predict(x_valid)

In [40]:
precision_score(y_valid, y_predict)

0.9325102880658436

In [41]:
recall_score(y_valid, y_predict)

0.7776252573781743

In [42]:
f1_score(y_valid, y_predict)

0.8480538922155688

In [43]:
roc_auc_score(y_valid, y_predict)

0.8587759986524571

可通过调整 k_n 参数， 距离权重 来提高recall——score

# Naive bayes model

In [45]:
x_train.shape, x_valid.shape

((11288, 200), (2822, 200))

In [59]:
nbm = GaussianNB()
nbm.fit(x_train.toarray(), y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [61]:
y_pred = nbm.predict(x_valid.toarray())

In [63]:
nbm.score(x_valid.toarray(), y_valid)

0.8497519489723601

In [64]:
precision_score(y_valid, y_pred)

0.927920463960232

In [65]:
recall_score(y_valid, y_pred)

0.7687028140013726

In [66]:
f1_score(y_valid, y_pred)

0.8408408408408409

In [68]:
roc_auc_score(y_valid, y_pred)

0.8524832751325545

训练速度快多了，准确度和knn差不多

# Logistic Regression model

In [69]:
lrm = LogisticRegression(n_jobs=-1)
lrm.fit(x_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1296: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)

In [70]:
lrm.score(x_valid, y_valid)

0.9447200566973778

In [71]:
y_pre = lrm.predict(x_valid)

In [73]:
precision_score(y_valid, y_pre)

0.9744711889132021

In [74]:
recall_score(y_valid, y_pre)

0.9169526424159231

In [75]:
f1_score(y_valid, y_pre)

0.9448373408769448

In [76]:
roc_auc_score(y_valid, y_pre)

0.9456558083874488

逻辑回归结果最好，比knn 与 bayes 高很多，而且速度快

# 测试模型

In [79]:
knn.score(X_test, Y_test)

0.8718875502008032

In [81]:
y_pred = knn.predict(X_test)

In [82]:
precision_score(Y_test, y_pred)

0.9306742640075973

In [83]:
recall_score(Y_test, y_pred)

0.799347471451876

In [84]:
f1_score(Y_test, y_pred)

0.8600263273365512

In [85]:
roc_auc_score(Y_test, y_pred)

0.870797153447457

In [87]:
nbm.score(X_test.toarray(), Y_test)

0.8710843373493976

In [88]:
y_pred = nbm.predict(X_test.toarray())

In [89]:
precision_score(Y_test, y_pred)
recall_score(Y_test, y_pred)
f1_score(Y_test, y_pred)
roc_auc_score(Y_test, y_pred)

0.8699569713176534

In [90]:
lrm.score(X_test, Y_test)

0.946987951807229

In [92]:
y_pred = lrm.predict(X_test)

In [93]:
precision_score(Y_test, y_pred)

0.9773123909249564

In [94]:
recall_score(Y_test, y_pred)

0.9135399673735726

In [95]:
f1_score(Y_test, y_pred)

0.9443507588532883

In [96]:
roc_auc_score(Y_test, y_pred)

0.946485173560204

利用逻辑回归判断一篇文章

In [105]:
index = 765

In [106]:
tester = news_corpus.loc[index]

In [107]:
tester

id                                                     88852
author                                            柳聪_BJS3376
source                                               新京报即时新闻
content    （原标题：被河北省教育厅责令“切割”后 衡中系两校仍联合对外宣传 |沸点）\r\n图片来源于...
feature    {"type":"社会","site":"网易","commentNum":"16","jo...
title                                 被责令＂切割＂后 衡中系两校仍联合对外宣传 
url        http://news.163.com/17/0620/13/CNCKF4QL00018AO...
Name: 765, dtype: object